# UE boss damage simulation
This attempts to simulate hitting bosses with certain lt configurations.

For now it supports Overkill, Tanya, The collector, Simba and leopard. 

It doesn't consider lt crits since we don't know the proc chance for that and we assume a baseline of crits dealing double damage without any further investment into crit dmg.

In [5]:
BASE_CRIT_DAMAGE = 100 # this means crit deal double dmg by default

In [6]:
import random
import pandas as pd
from typing import List, Tuple, Dict

def simulate_damage(
    base_damage: float = 10.0,
    percent_damage_mod: float = 0.0,
    percent_boss_damage_mod: float = 0.0,
    crit_chance: float = 25.0,
    crit_damage: float = 50.0,
    num_hits: int = 10000,
    use_ok: bool = False,
    ok_proc_chance: float = 10.0,
    use_collector: bool = True,
    collector_multiplier: float = 3.0,
    use_tanya: bool = False,
    tanya_proc_chance: float = 30,
    tanya_dmg: int = 50,
    use_caio: bool = False,
    caio_proc_chance: float = 15.0,
    use_leopard: bool = False,
    leopard_multiplier: float = 2.0,
    use_simba: bool = False,
    simba_proc_chance: float = 4.0,
) -> Tuple[List[float], List[Dict[str, float]]]:
    """
    Simulate damage for a series of hits, with optional critical hits,
    overkill procs, Tanya procs, collector procs, Caio procs, simba procs and leopard procs.
    """
    hits = []
    proc_data = []

    for _ in range(num_hits):
        # Calculate the regular damage with generic damage and boss damage modifiers
        regular_damage = base_damage * (1 + percent_damage_mod / 100) * (1 + percent_boss_damage_mod / 100)

        is_crit = False
        collector_proc_occurred = False
        tanya_proc_occurred = False
        ok_proc_occurred = False
        ok_proc_depth = 0
        caio_proc_occurred = False
        leopard_proc_occured = False
        simba_proc_occured = False

        # Determine if the hit is a critical hit
        if random.random() < crit_chance / 100:
            is_crit = True
            crit_extra_damage = regular_damage * ((crit_damage + BASE_CRIT_DAMAGE) / 100)
            damage = regular_damage + crit_extra_damage
            
            if use_ok:
                # Handle Overkill chain calculation
                multiplier = 1
                while random.random() < ok_proc_chance / 100:
                    ok_proc_occurred = True
                    ok_proc_depth += 1
                    multiplier *= 2
                    if multiplier >= 1000:
                        break
                crit_extra_damage *= multiplier
                damage = regular_damage + crit_extra_damage

            if use_tanya:
                if random.random() < tanya_proc_chance / 100:
                    tanya_proc_occurred = True
                    damage *= 1 + tanya_dmg / 100

            # Handle the collector proc (only on crit)
            if use_collector:
                if random.random() < 15 / 100:
                    collector_proc_occurred = True
                    damage *= collector_multiplier
        else:
            damage = regular_damage
            
        if use_caio:
            if random.random() < caio_proc_chance / 100:
                caio_proc_occurred = True
                damage *= 2

        # handle sk procs
        if use_leopard:
            if random.random() < 10 / 100:
                damage *= leopard_multiplier
                leopard_proc_occured = True

        if use_simba:
            if not leopard_proc_occured:
                 if random.random() < simba_proc_chance / 100:
                    damage *= 3
                    simba_proc_occured = True

        hits.append(damage)
        # Build the proc_data dictionary conditionally
        proc_entry = {
            'Damage': damage,
            'IsCrit': is_crit,
        }

        if use_collector:
            proc_entry['CollectorProcOccurred'] = collector_proc_occurred
        
        if use_ok:
            proc_entry['OKProcOccurred'] = ok_proc_occurred
            proc_entry['OKProcDepth'] = ok_proc_depth
        
        if use_tanya:
            proc_entry['TanyaProcOccurred'] = tanya_proc_occurred
            
        if use_caio:
            proc_entry['CaioProcOccurred'] = caio_proc_occurred

        if use_leopard:
            proc_entry['LeopardProcOccurred'] = leopard_proc_occured

        if use_simba:
            proc_entry['SimbaProcOccurred'] = simba_proc_occured

        proc_data.append(proc_entry)

    return hits, proc_data


## MMB simulation, 100% crit chance with Tanya and Collector

In [7]:
#Example usage
num_hits = 100000
base_damage = 10
crit_chance = 100
crit_damage = 100
use_ok = True
ok_proc_chance = 24
percent_damage_mod = 87
use_collector = True
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = True
tanya_proc_chance = 30
tanya_dmg = 110

# num_hits = 100000
# base_damage = 10
# crit_chance = 100
# crit_damage = 156
# use_ok = False
# ok_proc_chance = 24
# percent_damage_mod = 112
# use_collector = True
# collector_multiplier = 3.5  # Multiplier for the entire hit
# use_tanya = True
# tanya_proc_chance = 30
# tanya_dmg = 110

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    use_collector=use_collector,
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_proc_chance=tanya_proc_chance,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      133.804680
std       205.665052
min        56.100000
25%        56.100000
50%        93.500000
75%       117.810000
max     40251.750000


Damage     IsCrit  CollectorProcOccurred  OKProcOccurred  OKProcDepth  TanyaProcOccurred
56.100     True    False                  False           0            False                45212
117.810    True    False                  False           0            True                 19297
93.500     True    False                  True            1            False                10764
196.350    True    True                   False           0            False                 7947
                   False                  True            1            True                  4756
412.335    True    True                   False           0            True                  3438
168.300    True    False                  True            2            False                 2606
327.250    True    True           

## Jared, OK, Tanya, Collector, Caio

In [8]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 39.7 + 55
crit_damage = 45.8 + 55
use_ok = True
ok_proc_chance = 24
percent_damage_mod = 100
use_collector = True
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = True
tanya_proc_chance = 60
tanya_dmg = 70
use_caio = True

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    use_collector=use_collector,
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_proc_chance=tanya_proc_chance,
    tanya_dmg=tanya_dmg,
    use_caio=use_caio)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      166.584961
std       221.640678
min        20.000000
25%        60.160000
50%       102.272000
75%       200.640000
max     15411.928000


Damage     IsCrit  CollectorProcOccurred  OKProcOccurred  OKProcDepth  TanyaProcOccurred  CaioProcOccurred
102.272    True    False                  False           0            True               False               31224
60.160     True    False                  False           0            False              False               20909
170.544    True    False                  True            1            True               False                7527
204.544    True    False                  False           0            True               True                 5498
357.952    True    True                   False           0            True               False                5348
100.320    True    False                  True            1            False              False                5064
20.000

## MMB simulation, 100% crit chance with OK and Collector

In [9]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 39.7 + 55
crit_damage = 45.8 + 55
use_ok = True
ok_proc_chance = 24
percent_damage_mod = 5.4
use_collector = False
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = False
tanya_proc_chance = 60
tanya_dmg = 70

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    use_collector=use_collector,
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_proc_chance=tanya_proc_chance,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean       39.905282
std        42.229057
min        10.540000
25%        31.704320
50%        31.704320
75%        31.704320
max      5428.605920


Damage      IsCrit  OKProcOccurred  OKProcDepth
31.70432    True    False           0              71957
52.86864    True    True            1              17244
10.54000    False   False           0               5307
95.19728    True    True            2               4166
179.85456   True    True            3                997
349.16912   True    True            4                247
687.79824   True    True            5                 62
1365.05648  True    True            6                 15
2719.57296  True    True            7                  4
5428.60592  True    True            8                  1
Name: count, dtype: int64


## MMB simulation, 100% crit chance with Yugo and Collector

In [10]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 100
crit_damage = 279
use_ok = False
ok_proc_chance = 24
percent_damage_mod = 122
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = False
tanya_dmg = 110

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean      146.012708
std        94.727662
min       106.338000
25%       106.338000
50%       106.338000
75%       106.338000
max       372.183000


Damage   IsCrit  CollectorProcOccurred
106.338  True    False                    85076
372.183  True    True                     14924
Name: count, dtype: int64


In [11]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 47
crit_damage = 152
percent_damage_mod = 98
collector_multiplier = 3.5  # Multiplier for the entire hit

# Initialize lists to store results from each run
all_hits = []
all_proc_data = []

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe().round(2))
print("\n")
print(df_proc_data.value_counts())

          Damage
count  100000.00
mean       55.75
std        57.32
min        19.80
25%        19.80
50%        19.80
75%        69.70
max       243.94


Damage   IsCrit  CollectorProcOccurred
19.800   False   False                    52854
69.696   True    False                    40016
243.936  True    True                      7130
Name: count, dtype: int64
